# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key
gmaps.configure(api_key = g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
output_data_file = "output_data/cities.csv"
types_df = pd.read_csv(output_data_file)
weather_data_df = types_df.drop(columns=["Unnamed: 0"])
weather_data_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,krasnoselkup,43,RU,1584981092,91,65.70,82.47,0.43,19.93
1,nikolskoye,8,RU,1584981093,40,59.70,30.79,33.80,6.71
2,oistins,40,BB,1584981093,58,13.07,-59.53,86.00,20.80
3,vila franca do campo,20,PT,1584981094,55,42.03,-8.16,66.20,6.93
4,ekuvukeni,92,ZA,1584981094,82,-28.47,30.16,65.23,5.91
...,...,...,...,...,...,...,...,...,...
555,eyl,53,SO,1584981160,78,7.98,49.82,82.87,11.50
556,juarez,1,MX,1584981015,25,31.73,-106.48,66.20,4.70
557,rodeo,75,US,1584981160,81,38.03,-122.27,55.40,9.17
558,redmond,75,US,1584981160,75,47.67,-122.12,46.00,8.05


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
locations = weather_data_df[["Lat", "Lng"]]

humidity = weather_data_df["Humidity"].astype(float)

fig = gmaps.figure()

heatmap_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                   dissipating=False, max_intensity=5,
                                   point_radius=1)

fig.add_layer(heatmap_layer)

fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
new_types_df = pd.DataFrame(weather_data_df, columns = ["City", "Max Temp", "Wind Speed", "Cloudiness"])

max_temp = (new_types_df["Max Temp"] <= 80) & (new_types_df["Max Temp"] > 70)
wind_speed = new_types_df["Wind Speed"] < 10
cloudiness = new_types_df["Cloudiness"] == 0

new_types_df = new_types_df.dropna()
new_types_df 

,City,Max Temp,Wind Speed,Cloudiness
0,krasnoselkup,0.43,19.93,43
1,nikolskoye,33.80,6.71,8
2,oistins,86.00,20.80,40
3,vila franca do campo,66.20,6.93,20
4,ekuvukeni,65.23,5.91,92
...,...,...,...,...
555,eyl,82.87,11.50,53
556,juarez,66.20,4.70,1
557,rodeo,55.40,9.17,75
558,redmond,46.00,8.05,75


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = pd.DataFrame(weather_data_df, columns=["City", "Country", "Lat", "Lng"])

hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
0,krasnoselkup,RU,65.70,82.47,
1,nikolskoye,RU,59.70,30.79,
2,oistins,BB,13.07,-59.53,
3,vila franca do campo,PT,42.03,-8.16,
4,ekuvukeni,ZA,-28.47,30.16,
...,...,...,...,...,...
555,eyl,SO,7.98,49.82,
556,juarez,MX,31.73,-106.48,
557,rodeo,US,38.03,-122.27,
558,redmond,US,47.67,-122.12,


In [7]:
import json
from pprint import pprint

params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

for index, row in hotel_df.iterrows():
    
    lat = row["Lat"]
    lng = row["Lng"]

    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(base_url, params=params).json()
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 0: krasnoselkup.
Closest hotel is Krasnosel'kupnefteprodukt.
------------
Retrieving Results for Index 1: nikolskoye.
Closest hotel is Tourist House - Sablino.
------------
Retrieving Results for Index 2: oistins.
Closest hotel is Butterfly Beach Hotel.
------------
Retrieving Results for Index 3: vila franca do campo.
Closest hotel is Hotel Castrum Villae.
------------
Retrieving Results for Index 4: ekuvukeni.
Closest hotel is Sibeko Resident.
------------
Retrieving Results for Index 5: punta arenas.
Closest hotel is Vyjannex boarding house.
------------
Retrieving Results for Index 6: cudahy.
Closest hotel is Quality Inn & Suites At The Bicycle Casino.
------------
Retrieving Results for Index 7: barrow.
Closest hotel is King Eider Inn.
------------
Retrieving Results for Index 8: rikitea.
Closest hotel is Pension Maro'i.
------------
Retrieving Results for Index 9: pochutla.
Closest hotel is Hotel Posada San Jose.
------------
Retrieving Results for In

In [8]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""

hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [9]:
# Add marker layer ontop of heat map
locations = hotel_df[["Lat", "Lng"]]
markers = gmaps.marker_layer(locations)
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=hotel_info)

fig.add_layer(markers)
fig.add_layer(hotel_layer)
fig


Figure(layout=FigureLayout(height='420px'))